# Yelp API Call for Sushi places in US

## Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [3]:
# Load data for cities and states
path = r"C:\Users\kdelfino\Documents\GitHub\Sushi-Place-Finder\us_states_cities\uscities.csv"
us_cities_states = pd.read_csv(path)


In [4]:
# Filter only cities and states
us_cities_states = us_cities_states[['city','state_id']]
us_cities_states

,city,state_id
0,New York,NY
1,Los Angeles,CA
2,Chicago,IL
3,Miami,FL
4,Dallas,TX
...,...,...
30839,Ironville,PA
30840,Newkirk,NM
30841,Falcon Village,TX
30842,Hidden Lakes,OH


In [5]:
# Get Cities and State list 
us_cities_states_list = []
for i in range(len(us_cities_states['city'])):
    city = us_cities_states.iloc[i][0]
    state = us_cities_states.iloc[i][1]
    city_state = f"{city},{state}"
    us_cities_states_list.append(city_state)

us_cities_states_list

['New York,NY',
 'Los Angeles,CA',
 'Chicago,IL',
 'Miami,FL',
 'Dallas,TX',
 'Houston,TX',
 'Philadelphia,PA',
 'Atlanta,GA',
 'Washington,DC',
 'Boston,MA',
 'Phoenix,AZ',
 'Detroit,MI',
 'Seattle,WA',
 'San Francisco,CA',
 'San Diego,CA',
 'Minneapolis,MN',
 'Brooklyn,NY',
 'Tampa,FL',
 'Denver,CO',
 'Queens,NY',
 'Baltimore,MD',
 'Las Vegas,NV',
 'St. Louis,MO',
 'Portland,OR',
 'Riverside,CA',
 'Orlando,FL',
 'Sacramento,CA',
 'San Juan,PR',
 'San Antonio,TX',
 'San Jose,CA',
 'Pittsburgh,PA',
 'Cincinnati,OH',
 'Manhattan,NY',
 'Cleveland,OH',
 'Indianapolis,IN',
 'Austin,TX',
 'Kansas City,MO',
 'Columbus,OH',
 'Charlotte,NC',
 'Virginia Beach,VA',
 'Bronx,NY',
 'Milwaukee,WI',
 'Providence,RI',
 'Jacksonville,FL',
 'Salt Lake City,UT',
 'Nashville,TN',
 'Raleigh,NC',
 'Memphis,TN',
 'Louisville,KY',
 'Richmond,VA',
 'Buffalo,NY',
 'Oklahoma City,OK',
 'Bridgeport,CT',
 'New Orleans,LA',
 'Fort Worth,TX',
 'Hartford,CT',
 'Tucson,AZ',
 'Honolulu,HI',
 'McAllen,TX',
 'Omaha,NE',


## Set API Connection

In [9]:
# Load Credentials
with open(r'/Users/kdelfino/.secret/yelp_api.json') as f: 
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [10]:
# Instantiate API 
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

## Functions

In [14]:
# Text colors for visuals
reset_color = "\033[0m"
red = "\033[0;31;40m[!]" + reset_color
green = "\033[0;32;40m[i]" + reset_color
yellow = "\033[0;33;40m[i]" + reset_color


# Custom function to create/delete JSON file
def json_file_maker(JSON_FILE, delete_if_exist=False):

    # Check if JSON file exists
    file = os.path.isfile(JSON_FILE)

    # If file exists
    if file: 

        # Check if user wants to delete if exists 
        if delete_if_exist:
            print(red+f" {JSON_FILE} Already Exist. Deleting previous file. . . ")
            # Delete prev file
            os.remove(JSON_FILE) 
            # Create new JSON file by calling the same function (recussive)
            json_file_maker(JSON_FILE,delete_if_exist=False)
            print(green+ f" {JSON_FILE} file has been created!")

        else: 
            print(red + f" {JSON_FILE} Already Exist."+reset_color)      

    # If no existing file
    else:

        print(yellow+f" {JSON_FILE} is a new file, Creating empty folder. . . "+reset_color)

        # Get folder name
        folder = os.path.dirname(JSON_FILE)

        if len(folder)>0:
            # Create the folder
            os.makedirs(folder,exist_ok=True)

        with open(JSON_FILE,'w') as f:
            json.dump([],f)
        
        return print(green + f" New JSON file {JSON_FILE} has been created!"+reset_color)

In [15]:
# Create custom fuction to run query 
def run_query(json_file ,location:str, term:str, write_mode = False):

    # Load json file
    with open(json_file,'r') as f:
        previous_results = json.load(f)

    # Set offset based on prev results
    n_results = len(previous_results)


    # Run query and store in results 
    results = yelp_api.search_query(location=LOCATION,
                                    term= TERM,
                                    offset= n_results)

    total_results = results['total']
    per_page = len(results['businesses'])
    total_pages = math.ceil((total_results-n_results)/per_page)

    if not write_mode:
        return print(green+f"{total_results} total results with {per_page} per page and {total_pages} total pages")
    
    elif write_mode:

        # run query and write the results to json file
        for i in tqdm_notebook(range(1,total_pages+1)):
            # Load json file
            with open(json_file,'r') as f:
                previous_results = json.load(f)

            # Set breakpoint
            limit = 1000
            current = n_results+per_page
            if current > limit:
                print(red+f"Exeeded {limit} API calls limit, Query shutting down.")
                break

            # Set offset based on prev results
            n_results = len(previous_results)

            # Run query and store in results 
            results = yelp_api.search_query(location=LOCATION,
                                            term= TERM,
                                            offset= n_results)
            # Update results
            previous_results.extend(results['businesses'])

            # Save current results to json file
            with open(JSON_FILE,'w') as f:
                json.dump(previous_results,f)
            
            time.sleep(.2)
        
        return print(green+"Query Complete! Final Results: "+"\033[0;32;40m"+f"{results['total']}"+reset_color)

In [17]:
# Run Query for all cities in us
for location in us_cities_states_list:
    # Set API call params
    LOCATION = location
    TERM = "Sushi"

    # Create path for results 
    JSON_FILE = f"Data/results_in_progress_{LOCATION.split(',')[0]}_{TERM.replace(' ','_')}.json"
    
    # Create json file 
    json_file_maker(JSON_FILE,delete_if_exist=False)

    # try if our func will catch json file already exists 
    json_file_maker(JSON_FILE,delete_if_exist=False)

    # Run query and save results into json file 
    run_query(JSON_FILE,LOCATION,TERM,write_mode=True)

[!] Data/results_in_progress_New York_Sushi.json Already Exist.
[!] Data/results_in_progress_New York_Sushi.json Already Exist.


YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.